<a href="https://colab.research.google.com/github/debaghtk/finetuning-poc/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformers datasets torch accelerate pprint

download model tokenizer and dataset

In [ ]:
from transformers import AutoModelForCausalLM as AutoModel, AutoTokenizer
from datasets import load_dataset

model_name = "bigscience/bloomz-560m"

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("ought/raft", "twitter_complaints")

doing inference like TDD

In [ ]:
inputs = tokenizer(
    "Tweet text: @nationalgridus I have no water and the bill is current and paid. Can you do something about it? Label:",
    padding=True,
    max_length=64,
    truncation=True,
    return_tensors="pt",
)

output = model.generate(**inputs)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

preprocessing of the data
- add text_label key, values - complaint, no complaint

In [ ]:
dataset = load_dataset("ought/raft", "twitter_complaints")
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"label_ids": tokenizer(classes[x["Label"]], pad_to_max_length=True, max_length=64)["input_ids"]},
)
dataset = dataset.map(
    lambda x: {"model_input": f'Tweet text: {x["Tweet text"]}, Label:'},
)
dataset = dataset.remove_columns(["Label"])
# dataset = dataset.remove_columns(["Labels"])
# dataset = dataset.remove_columns(["labels"])
# dataset = dataset.remove_columns(["label"])
for i in range(len(dataset["test"])):
  if len(dataset["test"][i]["label_ids"])!=64:
    print("gadbad")

tokenize data

In [ ]:
dataset = dataset.map(
    lambda x: tokenizer(x["model_input"], pad_to_max_length=True, max_length=64),
    batched=True,
)
for i in range(len(dataset["train"])):
  if len(dataset["train"][i]["input_ids"])!=len(dataset["train"][i]["label_ids"]):
    print("gadbad")

split data into train, test, validation

In [ ]:
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

training arguments

In [ ]:
from transformers import TrainingArguments, trainer
import torch

model.to(torch.device("cpu"))
training_args = TrainingArguments(
    num_train_epochs=3,
    output_dir="results",
)

trainer = trainer.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()